In [1]:
# Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Write import Write
from IO.Read import Read
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

In [2]:
spark = SparkSession.builder.appName('melanoma').getOrCreate()

In [ ]:
#spark.stop()

In [ ]:
#path = '/home/george/diplwmatikh/code/maf/maf = 0.05/assocs/'
#pathPatients = '/home/george/diplwmatikh/data/patients/Patients.txt'
pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'

# Read ASSOCS

In [ ]:

start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(path)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(CHR=(p[0].strip()),SNP=p[1].strip(),BP=p[2].strip(),A1=p[3].strip(),F_A=p[4].strip()
                                   ,F_U=p[5].strip(),A2=p[6].strip(),CHISQ=p[7].strip(),P=(p[8].strip())
                                      ,OR=p[9].strip()))
schemaString = "CHR SNP BP A1 F_A F_U A2 CHISQ P OR"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaAssocs = spark.createDataFrame(inferRow,schema)
schemaAssocs.createOrReplaceTempView("Assocs")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

# Read Patients

In [ ]:
start_time = time.time()
#path = '/home/george/snp/'
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
log_txt = sc.textFile(pathPatients)
sc.setLogLevel("WARN")
header = log_txt.first()
log_txt = log_txt.filter(lambda line: line != header)
temp_var = log_txt.map(lambda k: k.split())

inferRow = temp_var.map(lambda p: Row(eid=(p[0].strip()),sex=p[1].strip(),birth_year=p[2].strip(),cases=p[3].strip()))
schemaString = "eid sex birth_year cases"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schemaPatients = spark.createDataFrame(inferRow,schema)
schemaPatients.createOrReplaceTempView("Patients")
#df=temp_var.toDF(header.split())
#log_df.select(log_df['field2']).show()
#log_df.filter(log_df['field2']=='1').count()
elapsed_time = time.time() - start_time
print(elapsed_time)
#df =spark.read.csv(path,header=True,inferSchema=True,escape="\t")

# TO REMOVE NA

In [ ]:
schemaAssocs = schemaAssocs.where("OR != 'NA'").drop()

In [ ]:
schemaAssocs.createOrReplaceTempView("Assocs")


In [ ]:
sqlDF = spark.sql("SELECT * FROM Assocs WHERE SNP=='.' ")
sqlDF.show()

In [ ]:
schemaPatients.show()

# STRING INDEXING

In [ ]:
snp_indexer = StringIndexer(inputCol='SNP',outputCol='SNPIndex')
id_indexer = StringIndexer(inputCol='eid',outputCol='eidIndex')

In [ ]:
Patients =id_indexer.fit(schemaPatients).transform(schemaPatients)

In [ ]:
Patients.show()

In [ ]:
Assocs = snp_indexer.fit(schemaAssocs).transform(schemaAssocs)

In [ ]:
Assocs.show()

# GET IDS FROM PATINETS AND SNP

In [ ]:
def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    yield ids

In [ ]:
ids={}
test= Patients.select('eid')
#idss = test.toPandas().to_dict(orient='list')
snps = Assocs.select('SNPIndex')
idsa = snps.toPandas().to_dict(orient='list')
ids['patients'] = setIdToName(test.toPandas().to_dict(orient='list')['eid'])
ids['snps'] = idsa['SNPIndex']

In [ ]:
X, Y = read.readSnpsCode(patients,ids)

In [ ]:
test.toPandas().to_dict(orient='list')['eid']

# TESTS

In [12]:
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp1.txt'

pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp1.txt'

data = spark.read.csv(pathSnp,inferSchema=True,header=True)

In [13]:
features = []
for i in data.columns:
    if 'rs'  in i:
        features.append(i)
print(len(features))

7798


In [14]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

In [15]:
assempler = VectorAssembler(inputCols=features,outputCol='features')

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression


In [17]:
data=data.withColumnRenamed('TARGET','label')

log_reg = LogisticRegression(featuresCol='features',labelCol='label')


In [18]:
output = assempler.transform(data)

In [19]:
final_data = output.select('features','label')

In [ ]:
#pipeline = Pipeline(stages=[output, final_data, log_reg])

In [ ]:
train_data,test_data = final_data.randomSplit([0.9,0.1])

train_data=train_data.withColumnRenamed('TARGET','label')
test_data=test_data.withColumnRenamed('TARGET','label')

In [ ]:
lr_model = log_reg.fit(train_data)

In [ ]:
results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
AUC

In [ ]:
#results = results.select('prediction','label')

In [ ]:
from pyspark.ml.stat import Correlation
final_data1 = final_data.select('features')
corr = Correlation.corr(final_data1, "features").head()
corr[0].values[1]

In [ ]:
corr[0].values

In [75]:
snpsCount = {}
for i in range(len(features)):
    snpsCount[i] = 0
    
for i in range(len(features)):
    for j in range(i+1, len(features)):
        if corr[0].values[j] - 0.7 <= 1e-10:

            snpsCount[i] = snpsCount[i] + 1
            snpsCount[j] = snpsCount[j] + 1
            


In [76]:
count = 0
for i in snpsCount.keys():
        
    if snpsCount[i] >= 100 * (len(features)-1) / 100 and snpsCount[i] <= 100 * (len(features)-1) / 100:
        #snpsRed.append(i)
        count += 1

In [77]:
count

7624

In [68]:
'''from pyspark.ml.evaluation import RegressionEvaluator

#lr = LinearRegression(maxIter=maxIteration)
modelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='r2')

paramGrid = ParamGridBuilder().build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=modelEvaluator,
                          numFolds=10)

cvModel = crossval.fit(train_data)
cross_results = cvModel.transform(test_data)
results = cross_results.select('prediction','label')
results.show(10)'''

"from pyspark.ml.evaluation import RegressionEvaluator\n\n#lr = LinearRegression(maxIter=maxIteration)\nmodelEvaluator=RegressionEvaluator(predictionCol='prediction', labelCol='label', metricName='r2')\n\nparamGrid = ParamGridBuilder().build()\n\ncrossval = CrossValidator(estimator=pipeline,\n                          estimatorParamMaps=paramGrid,\n                          evaluator=modelEvaluator,\n                          numFolds=10)\n\ncvModel = crossval.fit(train_data)\ncross_results = cvModel.transform(test_data)\nresults = cross_results.select('prediction','label')\nresults.show(10)"

In [ ]:
'''from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.linalg import Vectors
results1 = results.select('prediction','label')
results1 = results1.rdd
results1 = results1.map(lambda lp: (lp.features, lp.label))
metrics = MulticlassMetrics(results1)'''